**MILESTONE 4**

**Ali Wehbi - Dina Younes - Reeda Al Saintbai - Shaza Fakih - Youssef Jaafar**




In [ ]:

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/iseardataset.csv")
print(df.head())


     label                                               text Unnamed: 2
0      joy  On days when I feel close to my partner and ot...        NaN
1     fear  Every time I imagine that someone I love or I ...        NaN
2    anger  When I had been obviously unjustly treated and...        NaN
3  sadness  When I think about the short time that we live...        NaN
4  disgust  At a gathering I found myself involuntarily si...        NaN


In [ ]:
#Removing last column
data = df[['label','text']] 
print(data.head())

     label                                               text
0      joy  On days when I feel close to my partner and ot...
1     fear  Every time I imagine that someone I love or I ...
2    anger  When I had been obviously unjustly treated and...
3  sadness  When I think about the short time that we live...
4  disgust  At a gathering I found myself involuntarily si...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516 entries, 0 to 7515
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       7516 non-null   object
 1   text        7516 non-null   object
 2   Unnamed: 2  3 non-null      object
dtypes: object(3)
memory usage: 176.3+ KB




```
`# This is formatted as code`
```

#Preprocessing of data#

In [ ]:
import re
import numpy as np
import itertools
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

english_stopwords = stopwords.words('english')

def PreProcessing(text):
  
  text = text.lower() #convert to lower case
  text = re.sub(r'[^a-z]', ' ', text) #keep alphabetic words only
  tokens = word_tokenize(text) #create tokens of the text
  
  #print('\n')
 
  stemmer = PorterStemmer()
  stemmed = [stemmer.stem(word) for word in tokens] #stem words, i.e. return words into their root. 
  text = ' '.join(stemmed) #after preprocessing, create full text back
  #print("After preprocessing, before stopwords:")
  #print(text)
  #print('\n')
  # remove stopwords
  text = ' '.join([word for word in text.split() if word not in english_stopwords])
  #print("After stopwords:")
  #print('\n')
  
 
  return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data['text'] = data['text'].apply(PreProcessing)
data.head() 
print(data.head())

     label                                               text
0      joy  day feel close partner friend feel peac also e...
1     fear  everi time imagin someon love could contact se...
2    anger          obvious unjustli treat possibl elucid thi
3  sadness  think short time live relat period life think ...
4  disgust  gather found involuntarili sit next two peopl ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Feature Engineering#

In [ ]:
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = data.text.to_numpy()
#X = X.reshape(-1,1)
y = data.label.values
#y = y.reshape(-1,1)
#print(X)
#print(y)
#print(X.shape)
#print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=62)
"""x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)

y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)"""
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9) 
print(vectorizer)
train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)
#print(train_vectors)
#print(test_vectors)



TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.9, max_features=None,
                min_df=4, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


#Model Building#

In [ ]:
#import all the requirements 
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils


#specify the parameters of the NN model
output = 7
batch_size = 64
epochs = 20

#encode the output classes to be able to use in neural network i.e transform the labels into numbers 
#the encoding will be: 0:anger, 1:disgust, 2:fear, 3: guilt,
# 4: joy, 5: sadness, 6:shame 
from sklearn.preprocessing import LabelEncoder
def prepare_target(y):
  label = LabelEncoder()
  label.fit(y)
  y_encoded = label.transform(y)
  return y_encoded 

#prepare the data to test the NN model
y_train_encoded = prepare_target(y_train)
y_test_encoded = prepare_target(y_test)
Y_train = np_utils.to_categorical(y_train_encoded, output)


#build the model
model = Sequential()
#first hidden layer with 1000 neurons 
model.add(Dense(1000,input_shape= (train_vectors.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.5))  #to avoid overfitting

#second hidden layer with 500 neurons 
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.5))    #to avoid overfitting 

#third hidden layer with 50 neurons 
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))     #to avoid overfitting 

#last layers with 7 neurons, each neuron gives one output 
model.add(Dense(output))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print (model.summary())


#fit the model
model.fit(train_vectors, Y_train, batch_size= batch_size, epochs= epochs,verbose=2)

#test the model
y_predicted = model.predict_classes(test_vectors,batch_size=batch_size)

#print the accuracy, precision, f1score, and recall of the model
from sklearn.metrics import classification_report
print ("nDeep Neural Network - Test Classification Report")
print (classification_report(y_test_encoded,y_predicted))




Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1000)              1736000   
_________________________________________________________________
activation_12 (Activation)   (None, 1000)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 500)               500500    
_________________________________________________________________
activation_13 (Activation)   (None, 500)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)               

In [ ]:
def predict(model, vectorizer, text):
    text = PreProcessing(text)
    print(text)
    text_vectorized = vectorizer.transform([text])
    text_vectorized.sort_indices()
    return model.predict(text_vectorized)
text1 = "I am very happy today, i'm happy and joyful and i want to dance and sing just like Ahmad does."
text2 = "I feel awful today"
text3 = "I do not feel good today"
text4 = "I have a presentation tomorrow, im feeling nervous. I want to it to go well, but i might fail."
#print(predict(model,vectorizer,text1))
#print(predict(model,vectorizer,text2))

#the encoding will be: 0:anger, 1:disgust, 2:fear, 3: guilt, 4: joy, 5: sadness, 6:shame 
print(y_train[0:15])
print (y_train_encoded[0:15])

print(predict(model,vectorizer,text1))  
print(predict(model,vectorizer,text4))

['sadness' 'shame' 'disgust' 'joy' 'joy' 'anger' 'anger' 'joy' 'disgust'
 'shame' 'guilt' 'guilt' 'guilt' 'anger' 'fear']
[5 6 1 4 4 0 0 4 1 6 3 3 3 0 2]
veri happi today happi joy want danc sing like ahmad doe
[[2.7531841e-19 2.4563213e-18 2.1171378e-22 9.2299773e-25 1.0000000e+00
  6.6774900e-16 1.4327713e-17]]
present tomorrow im feel nervou want go well might fail
[[1.9647246e-02 1.2704944e-04 9.7093326e-01 6.1384059e-04 1.5036242e-04
  4.4420864e-03 4.0862611e-03]]
